In [1]:
import numpy as np
import tensorflow as tf

tf.logging.set_verbosity(tf.logging.INFO)

In [2]:
def cnn_model_fn(features, labels, mode):
    # Feature Extraction Layers

    # Input Layer (-1 indicates our batch size hyperparameter we can tune)
    input_layer = tf.reshape(features['x'], [-1, 28, 28, 1])
    # Convolution layer 1 
    # conv2d parameters: [batch_size, image_width, image_height, channels]
    # If kernel width and height are the same can do 'kernel_size=5'
    # Padding 'same' indicates that the kernel will do convolutions with 0s padded to the outside (i.e. same input dimension as input tensor)
    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=32,
        kernel_size=[5, 5],
        padding='same',
        activation=tf.nn.relu
    )

    # Pooling layer 1
    # Strides is (2) tiles skipped horizontally and vertically each sample (can do separate dimensions)
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2,2], strides=2)

    # Convolution and Pooling layer 2
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=64,
        kernel_size=[5, 5],
        padding='same',
        activation=tf.nn.relu
    )

    pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2,2], strides=2)

    # Classification Layers

    # Flatten layer (a long row vector of shape [batch_size x 3136])
    # For use in the next fully connected (dense) layer
    pool2_flat = tf.reshape(pool2, [-1,7 * 7 * 64])

    # Dense layer
    dense = tf.layers.dense(inputs=pool2_flat, units=1024, activation=tf.nn.relu)

    # Dropout (for regularization)
    # rate = 0.4 indicates that 40% nodes are randomly left out
    dropout = tf.layers.dropout(inputs=dense, rate=0.4, training=mode == tf.estimator.ModeKeys.TRAIN)

    # Logits (10 output classes)
    logits = tf.layers.dense(inputs=dropout, units=10)

    predictions = {
        # Generate Predictions
        "classes": tf.argmax(input=logits, axis=1),
        "probabilities": tf.nn.softmax(logits, name='softmax_tensor')
    }

    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, prediction=predictions)
    
    # Calculate Loss
    onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=10)
    loss = tf.losses.softmax_cross_entropy(onehot_labels=onehot_labels, logits=logits)

    # Configure Training Op (for TRAIN mode)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
        train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add metrics (for EVAL mode)
    eval_metric_ops = {
        "accuracy": tf.metrics.accuracy(
            labels=labels,
            predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
      mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)


In [6]:
def main(unused_argv):
    # Load training and eval data
    mnist = tf.contrib.learn.datasets.load_dataset("mnist")
    train_data = mnist.train.images # Returns np.array
    train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
    eval_data = mnist.test.images # Returns np.array
    eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)
    
    # The Estimator
    mnist_classifier = tf.estimator.Estimator(
        model_fn=cnn_model_fn, 
        model_dir='/tmp/mnist_convnet_model'
    )
    
    # Logging for predictions
    tensors_to_log = {"probabilities": "softmax_tensor"}
    logging_hook = tf.train.LoggingTensorHook(
        tensors=tensors_to_log,
        every_n_iter=50
    )
    
    # Train the model
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": train_data},
        y=train_labels,
        batch_size=100,
        num_epochs=None,
        shuffle=True
    )

    mnist_classifier.train(
        input_fn=train_input_fn,
        steps=100,
        #hooks=[logging_hook]
    )
    
    # Evaluate the model
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": eval_data},
        y=eval_labels,
        num_epochs=1,
        shuffle=True
    )
    eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
    print(eval_results)

    if __name__ == "__main__":
      tf.app.run()

In [9]:
tf.app.run()

Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/mnist_convnet_model', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /tmp/mnist_convnet_model/model.ckpt-3031
INFO:tensorflow:Saving checkpoints for 3032 into /tmp/mnist_convnet_model/model.ckpt.
INFO:tensorflow:loss = 0.447849, step = 3032
INFO:tensorflow:Saving checkpoints for 3131 into /tmp/mnist_convnet_model/model.ckpt.
INFO:tensorflow:Loss for final step: 0.701344.
INFO:tensorflow:Starting evaluation at 2017-12-26-00:48:30
INFO:ten

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from /tmp/mnist_convnet_model/model.ckpt-3631
INFO:tensorflow:Saving checkpoints for 3632 into /tmp/mnist_convnet_model/model.ckpt.
INFO:tensorflow:loss = 0.388668, step = 3632
INFO:tensorflow:Saving checkpoints for 3731 into /tmp/mnist_convnet_model/model.ckpt.
INFO:tensorflow:Loss for final step: 0.351151.
INFO:tensorflow:Starting evaluation at 2017-12-26-00:50:19
INFO:tensorflow:Restoring parameters from /tmp/mnist_convnet_model/model.ckpt-3731
INFO:tensorflow:Finished evaluation at 2017-12-26-00:50:23
INFO:tensorflow:Saving dict for global step 3731: accuracy = 0.9109, global_step = 3731, loss = 0.326257
{'accuracy': 0.9109, 'loss': 0.32625744, 'global_step': 3731}
Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Using default config.
INFO:tens

KeyboardInterrupt: 